<a href="https://colab.research.google.com/github/morprukop/SBALoan_ML_Project/blob/main/Notebooks/data_cleaning_notebook_ross.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project 2
### Due: Feb 22

Pick one of the datasets listed below. Then, do the following:
- Load it into a notebook using Pandas
- Find anything in it that needs to be cleaned. This could include rows with missing data, columns with data stored as the wrong type, dates that are stored as strings instead of dates, and more.
- Create the following items:
    - **A presentation:** Your presentation should be approximately 10 minutes in length, and cover the main topics. You can save code and nitty-gritty details for your notebook.
    - **A Jupyter notebook:** The notebook should include a mixture of Markdown cells describing your work, and code. There should be no scratch work, everything should be easy to read.
    - **An article on your LinkedIn page:** This should be comprehensible by a general, non-data science audience. Include graphs to back up your results. Include a link to your Github repository for this project.
- Each person on the team should create a new Github repository. It should have the following:
    - A nice title. Don't call it "Project 2". Make it something interesting so that someone not in this class would want to see what you did.
    - A README file explaining what you're doing. Make it look nice, use Markdown.
    - A folder titled `data` for your data. In this folder you should put the data. Also, once you've cleaned your data, include another file `cleaned_data.csv` which is the cleaned data.
    - A folder titled `notebooks` for your notebook. Give your notebook a nice filename.
    - A folder titled `presentations` for your presentation. Give your presentation a nice filename.
    - A folder titled `articles` for your LinkedIn article. 
- Fill out the [proposal template](proposal_template.ipynb) and email it to psavala@stedwards.edu. This is due on Monday, Sep 21 by midnight.

## Datasets:

Pick TWO questions from each of the bullet points below and address them.
 
- **Small business loans:** Use the [Small Business Administration (SBA) dataset](https://www.kaggle.com/mirbektoktogaraev/should-this-loan-be-approved-or-denied) to address the following questions:
    - What factors tend to cause a loan to be defaulted on?
    - What factors affect how much the loan is for?
    - Build a model to predict which approved loan applications will default on their loan.
    - _Target audience:_ A consultant who helps small businesses grow. In particular, you help guide them from initial idea until the point where they can qualify for a small business loan and be successful enough to not default on it.

# Data Loading & Imports

In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
from sklearn.linear_model import LinearRegression 
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df = pd.read_csv("/content/drive/Shareddrives/Loan_ Project2/Data/SBAnational.csv")

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


# First Look at The Data

In [ ]:
df.head()

,LoanNr_ChkDgt,Name,City,State,Zip,Bank,BankState,NAICS,ApprovalDate,ApprovalFY,Term,NoEmp,NewExist,CreateJob,RetainedJob,FranchiseCode,UrbanRural,RevLineCr,LowDoc,ChgOffDate,DisbursementDate,DisbursementGross,BalanceGross,MIS_Status,ChgOffPrinGr,GrAppv,SBA_Appv
0,1000014003,ABC HOBBYCRAFT,EVANSVILLE,IN,47711,FIFTH THIRD BANK,OH,451120,28-Feb-97,1997,84,4,2.0,0,0,1,0,N,Y,NaN,28-Feb-99,"$60,000.00",$0.00,P I F,$0.00,"$60,000.00","$48,000.00"
1,1000024006,LANDMARK BAR & GRILLE (THE),NEW PARIS,IN,46526,1ST SOURCE BANK,IN,722410,28-Feb-97,1997,60,2,2.0,0,0,1,0,N,Y,NaN,31-May-97,"$40,000.00",$0.00,P I F,$0.00,"$40,000.00","$32,000.00"
2,1000034009,"WHITLOCK DDS, TODD M.",BLOOMINGTON,IN,47401,GRANT COUNTY STATE BANK,IN,621210,28-Feb-97,1997,180,7,1.0,0,0,1,0,N,N,NaN,31-Dec-97,"$287,000.00",$0.00,P I F,$0.00,"$287,000.00","$215,250.00"
3,1000044001,"BIG BUCKS PAWN & JEWELRY, LLC",BROKEN ARROW,OK,74012,1ST NATL BK & TR CO OF BROKEN,OK,0,28-Feb-97,1997,60,2,1.0,0,0,1,0,N,Y,NaN,30-Jun-97,"$35,000.00",$0.00,P I F,$0.00,"$35,000.00","$28,000.00"
4,1000054004,"ANASTASIA CONFECTIONS, INC.",ORLANDO,FL,32801,FLORIDA BUS. DEVEL CORP,FL,0,28-Feb-97,1997,240,14,1.0,7,7,1,0,N,N,NaN,14-May-97,"$229,000.00",$0.00,P I F,$0.00,"$229,000.00","$229,000.00"


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 899164 entries, 0 to 899163
Data columns (total 27 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   LoanNr_ChkDgt      899164 non-null  int64  
 1   Name               899150 non-null  object 
 2   City               899134 non-null  object 
 3   State              899150 non-null  object 
 4   Zip                899164 non-null  int64  
 5   Bank               897605 non-null  object 
 6   BankState          897598 non-null  object 
 7   NAICS              899164 non-null  int64  
 8   ApprovalDate       899164 non-null  object 
 9   ApprovalFY         899164 non-null  object 
 10  Term               899164 non-null  int64  
 11  NoEmp              899164 non-null  int64  
 12  NewExist           899028 non-null  float64
 13  CreateJob          899164 non-null  int64  
 14  RetainedJob        899164 non-null  int64  
 15  FranchiseCode      899164 non-null  int64  
 16  Ur

In [ ]:
df.isna().sum()

LoanNr_ChkDgt             0
Name                     14
City                     30
State                    14
Zip                       0
Bank                   1559
BankState              1566
NAICS                     0
ApprovalDate              0
ApprovalFY                0
Term                      0
NoEmp                     0
NewExist                136
CreateJob                 0
RetainedJob               0
FranchiseCode             0
UrbanRural                0
RevLineCr              4528
LowDoc                 2582
ChgOffDate           736465
DisbursementDate       2368
DisbursementGross         0
BalanceGross              0
MIS_Status             1997
ChgOffPrinGr              0
GrAppv                    0
SBA_Appv                  0
dtype: int64

# Data Cleaning & Formatting

Renaming Columns

In [ ]:
df.columns = ["loan_id", "name", "city", "state", "zip", "bank", "bank_state", "naics", "date_appv", "appv_fy", "term", "num_emp", "is_new", "num_jobs", "num_retain", "franch", "urban_type", "is_revl", "is_ldoc", "date_dflt", "date_disb", "amt_disb", "amt_outst", "mis", "amt_chg_off", "amt_gr_appv", "amt_sba"]

Dropping N/A Values (Except for "date_dflt")

In [ ]:
df = df.dropna(subset=["name", "city", "state", "bank", "bank_state", "is_new", "is_revl", "is_ldoc", "date_disb", "mis"])

Cleaning columns in preparation for conversion to proper data types

In [ ]:
pd.options.mode.chained_assignment = None

In [ ]:
df["date_appv"] = pd.to_datetime(df["date_appv"], format="%d-%b-%y")
df["appv_fy"] = df["appv_fy"].astype(str).str.replace("[^0-9]", "", regex=True).astype(int)
df["is_new"] = df[df["is_new"].astype(int) > 0]["is_new"].map({2: True, 1: False})  # bool
df["urban_type"] = df["urban_type"] - 1
df["is_revl"] = df[df["is_revl"].astype(str).str.replace("[^NY]", "", regex=True) != ""]["is_revl"].map({"Y": 1, "N": 0})
df["is_ldoc"] = df[df["is_ldoc"].astype(str).str.replace("[^NY]", "", regex=True) != ""]["is_ldoc"].map({"Y": True, "N": False})  # bool
df["date_dflt"] = pd.to_datetime(df["date_dflt"], format="%d-%b-%y")
df["date_disb"] = pd.to_datetime(df["date_disb"], format="%d-%b-%y")
df["amt_disb"] = df["amt_disb"].astype(str).str.replace("[^0-9]", "", regex=True).astype(int)
df["amt_outst"] = df["amt_outst"].astype(str).str.replace("[^0-9]", "", regex=True).astype(int)
df["mis"] = df["mis"].map({"P I F": True, "CHGOFF": False})  # bool
df["amt_chg_off"] = df["amt_chg_off"].astype(str).str.replace("[^0-9]", "", regex=True).astype(int)
df["amt_gr_appv"] = df["amt_gr_appv"].astype(str).str.replace("[^0-9]", "", regex=True).astype(int)
df["amt_sba"] = df["amt_sba"].astype(str).str.replace("[^0-9]", "", regex=True).astype(int)

Dropping N/A values post-dtype cleaning

In [ ]:
df = df.dropna(subset=["is_new", "is_ldoc"])

Fill certain N/A values

In [ ]:
df = df.fillna({"is_revl": -1})

Converting remaining columns to proper data types after cleaning

In [ ]:
df[["is_new", "is_ldoc", "mis"]] = df[["is_new", "is_ldoc", "mis"]].astype(bool)
df["is_revl"] = df["is_revl"].astype(int)

Re-Indexing dataframe

In [ ]:
df.reset_index(level=0, inplace=True)
del df["index"]

# Looking at Data After Cleaning

### Data description

This dataset is from the U.S. Small Business Administration (SBA)

Each row represents one loan application from a borrower and each column represents:
- **loan_id**: Identifier – Primary key
- **name**: Name of the borrower
- **city**: City the borrower is from
- **state**: State the borrower is from
- **zip**: Zip code of the borrower
- **bank**: Bank the borrower is loaning from
- **bank_state**: State the bank is in
- **naics**: North American Industry Classification System code
- **date_appv**: Date SBA commitment issued
- **appv_fy**: Fiscal year of commitment
- **term**: Loan term in months
- **num_emp**: Number of business employees
- **is_new**: Is the loan for a new or existing business (True = new, False = existing)
- **num_jobs**: Number of jobs created
- **num_retain**: Number of jobs retained
- **franch**: Franchise code (if 0 or 1, then it's not a franchise)
- **urban_type**: Is the business urban or rural (1 = urban, 2 = rural, -1 = undefined)
- **is_revl**: Revolving line of credit (1 = yes, 0 = no, -1 = undefined)
- **is_ldoc**: Part of the LowDoc Loan Program (True = yes, False = no)
- **date_dflt**: The date when a loan is declared to be in default
- **date_disb**: Disbursement date
- **amt_disb**: Amount disbursed in pennies
- **amt_outst**: Gross amount outstanding in pennies
- **mis**: MIS status (True = paid in full, False = loan status charged off)
- **amt_chg_off**: Charged-off amount in pennies
- **amt_gr_appv**: Gross amount of loan approved by bank in pennies
- **amt_sba**: SBA’s guaranteed amount of approved loan in pennies

In [ ]:
df.head()

,loan_id,name,city,state,zip,bank,bank_state,naics,date_appv,appv_fy,term,num_emp,is_new,num_jobs,num_retain,franch,urban_type,is_revl,is_ldoc,date_dflt,date_disb,amt_disb,amt_outst,mis,amt_chg_off,amt_gr_appv,amt_sba
0,1000014003,ABC HOBBYCRAFT,EVANSVILLE,IN,47711,FIFTH THIRD BANK,OH,451120,1997-02-28,1997,84,4,True,0,0,1,-1,0,True,NaT,1999-02-28,6000000,0,True,0,6000000,4800000
1,1000024006,LANDMARK BAR & GRILLE (THE),NEW PARIS,IN,46526,1ST SOURCE BANK,IN,722410,1997-02-28,1997,60,2,True,0,0,1,-1,0,True,NaT,1997-05-31,4000000,0,True,0,4000000,3200000
2,1000034009,"WHITLOCK DDS, TODD M.",BLOOMINGTON,IN,47401,GRANT COUNTY STATE BANK,IN,621210,1997-02-28,1997,180,7,False,0,0,1,-1,0,False,NaT,1997-12-31,28700000,0,True,0,28700000,21525000
3,1000044001,"BIG BUCKS PAWN & JEWELRY, LLC",BROKEN ARROW,OK,74012,1ST NATL BK & TR CO OF BROKEN,OK,0,1997-02-28,1997,60,2,False,0,0,1,-1,0,True,NaT,1997-06-30,3500000,0,True,0,3500000,2800000
4,1000054004,"ANASTASIA CONFECTIONS, INC.",ORLANDO,FL,32801,FLORIDA BUS. DEVEL CORP,FL,0,1997-02-28,1997,240,14,False,7,7,1,-1,0,False,NaT,1997-05-14,22900000,0,True,0,22900000,22900000


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 882067 entries, 0 to 882066
Data columns (total 27 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   loan_id      882067 non-null  int64         
 1   name         882067 non-null  object        
 2   city         882067 non-null  object        
 3   state        882067 non-null  object        
 4   zip          882067 non-null  int64         
 5   bank         882067 non-null  object        
 6   bank_state   882067 non-null  object        
 7   naics        882067 non-null  int64         
 8   date_appv    882067 non-null  datetime64[ns]
 9   appv_fy      882067 non-null  int64         
 10  term         882067 non-null  int64         
 11  num_emp      882067 non-null  int64         
 12  is_new       882067 non-null  bool          
 13  num_jobs     882067 non-null  int64         
 14  num_retain   882067 non-null  int64         
 15  franch       882067 non-null  int6

# Exporting Dataframe (left commented for safety)

In [ ]:
# df.to_csv("sba_national_cleaned.csv")
# !cp "sba_national_cleaned.csv" "/content/drive/Shareddrives/Loan_ Project2/Data/"